In [1]:
import data_processing
import pandas as pd
import ipywidgets as widget
from ipywidgets import Layout
from IPython.display import HTML, display

with open('variables.css') as f:
    css = f.read().replace(';', ' !important;')
display(HTML('<style type="text/css">%s</style>'%css))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

items_layout = Layout(justify_content='center')
box_layout = Layout(display='flex',
                    flex_flow='row',                    
                    justify_content='center')

style_dataframe = [dict(selector="caption", props=[("color", "black"), 
                                                   ("border-radius", "5px"), 
                                                   ("background", "#f0f0f0"), 
                                                   ("font-weight", "bold"),
                                                   ("font-size", "20px")]), 
                   dict(selector="th.col_heading", props=[("color", "#5fba7d"),
                                                          ("border-radius", "3px"),
                                                          ("background", "#505050")]),
                   dict(selector="th.blank.level0", props=[("color", "#5fba7d"),
                                                          ("border-radius", "3px"),
                                                          ("background", "#505050")]),
                   dict(selector="th", props=[("text-align", "center")])]
                        
year_text = 'Año'
year_label = widget.HTML(value = f"<b style='color:white; font-family:tahoma; font-size:18px; border-radius: 3px 3px 3px 3px'>{year_text}</b>")
year_menu = widget.Dropdown(options=[2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030],
                            value=2021,                            
                            layout=items_layout)

month_text = 'Mes'
month_label = widget.HTML(value = f"<b style='color:white; font-family:tahoma; font-size:18px; border-radius: 3px 3px 3px 3px'>{month_text}</b>")
month_menu = widget.Dropdown(options=['Enero', 'Febrero', 'Marzo', 'Abril', 
                                     'Mayo', 'Junio', 'Julio', 'Agosto', 
                                     'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'],
                            value='Enero',
                            layout=items_layout)

out = widget.Output()

calculate = widget.Button(description='Filtrar',
                          layout=items_layout)
calculate.style.button_color = '#3aaf58'
def on_calculate_clicked(_):
    with out:
        month_number = data_processing.switcher_months[month_menu.value]
        show_salary_on_date(year_menu.value, month_number)
        show_worker_status_on_date(year_menu.value,month_number)
        show_jobs_on_date(year_menu.value,month_number) 
calculate.on_click(on_calculate_clicked)

errors = widget.Button(description='Errores',
                       layout=items_layout)
errors.style.button_color = '#e90303'
def on_errors_clicked(_):
    with out:        
        show_model_error()
errors.on_click(on_errors_clicked)

box1 = widget.Box([year_label, year_menu, month_label, month_menu, calculate, errors], layout=box_layout)

light_cols = ["Salario"]

def highlight_max(s):    
    if s.name in light_cols:
        is_max = s == s.max()
        return ['background-color: green' if v else '' for v in is_max]
    return [''] * len(s)

def highlight_min(s):        
    if s.name in light_cols:
        is_min = s == s.min()
        is_max = s == s.max()        
        mods = []
        for v in s:
            print(v)
            if v in is_min:
                mods.append('background-color: yellow')
            elif v in is_max:
                mods.append('background-color: green')
            else:
                mods.append('')       
        print(mods)
        return mods        
    return [''] * len(s)

def load_data():
    data_processing.traverse_dir()
    data_processing.asociate_status_with_jobs()
    
def show_salary_on_date(year, month):
    df_total_salary = pd.DataFrame(data_processing.total_salary_per_worker_date(year,month))
    df_total_salary = (df_total_salary.style
                                           #.apply(highlight_max, axis=0)
                                           #.apply(highlight_min, axis=0)
                                           #.bar(subset=['Salario Total'], color='#5fba7d')
                                           .set_table_styles(style_dataframe)
                                           .set_caption(str(year) + "/" + str(month))
                                           .format({'Salario Total':'$ {:.2f}'}))
    display(df_total_salary)

def show_worker_status():
    df_workers = pd.DataFrame(data_processing.status)
    df_workers = df_workers.drop(['Trabajado'], axis=1)
    df_workers = (df_workers.style
                                  #.apply(highlight_max)
                                  #.apply(highlight_min, axis=0)
                                  #.bar(subset=['Salario Escala', 'Salario'], color='#5fba7d')
                                  .set_table_styles(style_dataframe)
                                  .set_caption("CONTROL DEL TRABAJO REALIZADO MENSUAL")
                                  .format({'Salario Escala':'$ {:d}',
                                          'Salario':'$ {:.2f}'}))
    display(df_workers)
    
def show_worker_status_on_date(year, month):
    df_workers = pd.DataFrame(data_processing.status_per_worker_date(year, month))
    df_workers = (df_workers.style
                                  #.apply(highlight_max)
                                  #.apply(highlight_min, axis=0)
                                  #.bar(subset=['Salario Escala', 'Salario'], color='#5fba7d')
                                  .set_table_styles(style_dataframe)       
                                  .set_caption("CONTROL DEL TRABAJO REALIZADO MENSUAL")
                                  .format({'Salario Escala':'$ {:d}',
                                           'Salario':'$ {:.2f}'}))
    display(df_workers)

def show_jobs_on_date(year,month):
    t_worked = data_processing.job_done_per_worker_date(year,month)
    for e in t_worked:
        df_worked = pd.DataFrame(e[3])    
        df_worked = df_worked.dropna(axis=0)
        df_worked = (df_worked.style
                                    #.apply(highlight_max)
                                    #.apply(highlight_min, axis=0)
                                    #.bar(subset=['TIEMPO AL AIRE  (min)', 
                                    #'TIEMPO DE REALIZACIÓN  (hrs)', 
                                    #'Total de Programas Ejecutados ', 
                                    #'Salario', 
                                    #'Tarifa Salarial Horaria (hrs)',
                                    #'Importe por Programa'],
                                    #color=['red', '#5fba7d'])                                      
                                    #'Importe por Remoto',
                                    #'Importe por Interrupción',
                                    #'Importe por Simultaneidad',
                                    #'Nocturnidad',
                                    #'Descuento por Calidad',
                                    #'Importe por Programa'], 
                                    #color='#d65f5f')
                                    #.applymap(color_map)
                                    .set_table_styles(style_dataframe)
                                    .set_caption("Nombre: " + str(e[0]) + " C.I: " + str(e[1]) + " R.H: " + str(e[2]))
                                    .format({'TIEMPO DE REALIZACIÓN  (hrs)':'{:.2f}',
                                             'Salario':'$ {:d}', 
                                             'Tarifa Salarial Horaria (hrs)':'$ {:.2f}',
                                             'Importe por Remoto':'$ {:.2f}',
                                             'Importe por Interrupción':'$ {:.2f}',
                                             'Importe por Simultaneidad':'$ {:.2f}',
                                             'Nocturnidad':'$ {:.2f}',
                                             'Descuento por Calidad':'$ {:.2f}',
                                             'Importe por Programa':'$ {:.2f}'}))                                   
        display(df_worked)
        
def show_model_error():
    df_errors = pd.DataFrame(data_processing.non_valid)
    df_errors = (df_errors.style  
                                  .apply(lambda x: ["background: red" if v == "Error" else "" for v in x], axis=1)                                  
                                  .set_table_styles(style_dataframe)       
                                  .set_caption("ERRORES"))                                  
    display(df_errors)

load_data()
display(box1)
display(out)

Box(children=(HTML(value="<b style='color:white; font-family:tahoma; font-size:18px; border-radius: 3px 3px 3p…

Output()